In [ ]:
from datasets import load_dataset
from tqdm.auto import tqdm

/lustre/BIF/nobackup/holme003/progs/miniconda3/envs/deeplearning/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
train_data = load_dataset('Skylion007/openwebtext', streaming=True, split='train')

In [20]:
load_dataset('Skylion007/openwebtext', streaming=True)

IterableDatasetDict({
    train: IterableDataset({
        features: ['text'],
        num_shards: 21
    })
})

In [20]:
with open('openwebtext.input.txt','w') as fh:
  for d in tqdm(dataset):
    fh.write(d['text'].replace('\n',' ') + '\n')

8013769it [35:50, 3726.59it/s]


In [22]:
from torch.utils.data import IterableDataset, DataLoader, random_split
from datasets import load_dataset

class CharacterTokenizer:
    """Character level tokenizer that enumerates the first 256 unicode characters"""
    def __init__(self):
        self.vocab_size = 256
        self.encoding_dict = {chr(token_i): token_i for token_i in range(256)}
        self.decoding_dict = {token_i: chr(token_i) for token_i in range(256)}

    def __repr__(self):
        return f'CharacterTokenizer(vocab_size={self.vocab_size})'

    def get_vocab(self) -> dict[str, int]:
        return self.encoding_dict

    def encode(self, data: str) -> list[int]:
        """Convert text to tokens"""
        return [self.encoding_dict.get(char, -1) for char in data]

    def decode(self, tokens: list[int]) -> str:
        """Convert tokens to text"""
        return ''.join(self.decoding_dict.get(token, '<unk>') for token in tokens)

# Initialize an NaiveTokenizer
tokenizer = CharacterTokenizer()

class OWTDataset(IterableDataset):
  def __init__(self, tokenizer, context_size:int=64):
    super(OWTDataset).__init__()
    self.dataset = load_dataset('Skylion007/openwebtext', streaming=True, split='train')
    self.tokenizer = tokenizer
    self.context_size = context_size


  def __iter__(self):
    remainder = ''
    for d in self.dataset:
      text = self.tokenizer.encode(d['text'].replace('\n',' ') + '\n')
      if remainder:
        yield remainder + text[:self.context_size - len(remainder)]
      for pos in range(len(remainder), len(text), self.context_size):
        chunk = text[pos: pos+self.context_size]
        if len(chunk) < self.context_size:
          remainder = chunk
        else:
          yield chunk

train_dataset = OWTDataset(tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=32)
for data in train_dataloader:
  print(data)
  break

[tensor([ 80,  97, 114, 118, 100, 101, 103, 102,  32, 110,  97, 111, 108, 115,
        117, 116, 115, 101,  32, 117, 108, 110,  80, 100,  97, 115, 115, 111,
        111, 105,  32, 116]), tensor([111, 105, 111,  97, 101,  32, 104,  32,  65,  97, 110, 117,  32,  32,
        103, 101, 101,  97,  97, 101, 100, 103, 111,  32, 110, 116, 115, 114,
        115, 110, 109,  32]), tensor([114, 110, 109,  99,  99, 111, 116, 116, 105, 116,  32, 116, 116, 119,
        104, 114,  99, 109, 103, 115,  32,  32, 114,  83, 121,  32, 105, 116,
        112, 116, 101, 116]), tensor([116,  32,  32, 117, 105, 110,  46, 104, 100, 111, 109,  32, 111, 111,
         32,  32, 117,  32, 114, 116, 104, 116, 116,  97,  32, 111, 115, 101,
        105, 101, 100, 114]), tensor([ 45,  97,  97,  97, 115, 108,  32, 101,  32, 114, 101, 102,  32, 117,
        100,  97, 114, 114, 101, 101, 111, 111,  45, 116, 109, 102, 116, 114,
        116, 114, 105, 117]), tensor([ 97, 110,  32, 116, 105, 121,  32,  32,  97,  32, 100, 111, 1

In [ ]:
class CharacterTokenizer:
    """Character level tokenizer that enumerates the first 256 unicode characters"""
    def __init__(self):
        self.vocab_size = 256
        self.encoding_dict = {chr(token_i): token_i for token_i in range(256)}
        self.decoding_dict = {token_i: chr(token_i) for token_i in range(256)}

    def __repr__(self):
        return f'CharacterTokenizer(vocab_size={self.vocab_size})'

    def get_vocab(self) -> dict[str, int]:
        return self.encoding_dict

    def encode(self, data: str) -> list[int]:
        """Convert text to tokens"""
        return [self.encoding_dict.get(char, -1) for char in data]

    def decode(self, tokens: list[int]) -> str:
        """Convert tokens to text"""
        return ''.join(self.decoding_dict.get(token, '<unk>') for token in tokens)

# Initialize an NaiveTokenizer
tokenizer = CharacterTokenizer()
print(f'Untrained tokenizer: {tokenizer}')

# Encode a string
tokens = tokenizer.encode('Hi how are you')
print(f'{tokens = }')

# Decode the encoding
print(tokenizer.decode(tokens))

Untrained tokenizer: CharacterTokenizer(vocab_size=256)
tokens = [72, 105, 32, 104, 111, 119, 32, 97, 114, 101, 32, 121, 111, 117]
Hi how are you


In [ ]:
data = ['12345','abcd','qrstuvw']
context_size = 3

def it():
  remainder = ''
  for d in data:
    if remainder:
      chunk = remainder + d[:context_size - len(remainder)]
      yield chunk[:-1],chunk[1:]
    for pos in range(len(remainder), len(d), context_size):
      chunk = d[pos: pos+context_size + 1]
      if len(chunk) < context_size + 1:
        remainder = chunk
      else:
        yield chunk

for i in it:
  print(i)


TypeError: 'function' object is not iterable